In [11]:
if __package__ is None:
    import sys
    from os import path
    sys.path.append(path.dirname(path.dirname(path.abspath("Test.ipynb"))))

from FMLC.triggering import triggering
from FMLC.baseclasses import eFMU
from FMLC.stackedclasses import controller_stack, log_to_db
import time
from datetime import datetime
import pandas as pd
import numpy as np


In [12]:
class testcontroller1(eFMU):
    def __init__(self):
        self.input = {'a': None, 'b': None}
        self.output = {'c': None}
        self.init = True
    def compute(self):
        self.init= False
        self.output['c'] = self.input['a'] * self.input['b']


class testcontroller2(eFMU):
    def __init__(self):
        self.input = {'a': None, 'b': None}
        self.output = {'c': None}
        self.init = True

    def compute(self):
        self.init = False
        self.output['c'] = self.input['a'] * self.input['b']
        time.sleep(0.2)


class testcontroller3(eFMU):
    def __init__(self):
        self.input = {'a': None, 'b': None}
        self.output = {'c': None}
        self.init = True

    def compute(self):
        self.init = False
        self.output['c'] = self.input['a'] * self.input['b']
        time.sleep(1)

class testcontroller4(eFMU):
    def __init__(self):
        self.input = {'a': None, 'b': None}
        self.output = {'c': None}
        self.init = True

    def compute(self):
        self.init = False
        self.output['c'] = self.input['a'] * self.input['b']
        time.sleep(10)
controller = {}
controller['forecast1'] = {'fun':testcontroller1, 'sampletime':0}
controller['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controller['control1'] = {'fun':testcontroller1, 'sampletime':'mpc1'}
controller['forecast2'] = {'fun':testcontroller3, 'sampletime':0}
controller['forecast3'] = {'fun':testcontroller1, 'sampletime':0}
controller = controller_stack(controller, tz=-8, debug=True, parallel=True, timeout=2)

mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
controller.initialize(mapping)

for i in range(6):
    controller.query_control(time.time())
    time.sleep(1.5)

for df in controller.log_to_df().values():
    display(df)


,a,b,c,Logging
2020-09-01 16:09:38.225183964,NaN,NaN,NaN,Initialize
2020-09-01 21:12:56.442369938,10.0,4.0,40.0,None
2020-09-01 21:13:02.537092209,10.0,4.0,40.0,None


,a,b,c,Logging
2020-09-01 16:09:38.225183964,NaN,NaN,NaN,Initialize
2020-09-01 21:12:57.976781130,40.0,10.0,400.0,None
2020-09-01 21:13:04.071951151,40.0,10.0,400.0,None


,a,b,c,Logging
2020-09-01 16:09:38.225183964,NaN,NaN,NaN,Initialize
2020-09-01 21:12:59.490318060,400.0,40.0,16000.0,None


,a,b,c,Logging
2020-09-01 16:09:38.225183964,NaN,NaN,NaN,Initialize
2020-09-01 21:12:56.442369938,20.0,4.0,80.0,None
2020-09-01 21:12:59.490318060,20.0,4.0,80.0,None
2020-09-01 21:13:02.537092209,20.0,4.0,80.0,None


,a,b,c,Logging
2020-09-01 16:09:38.225183964,NaN,NaN,NaN,Initialize
2020-09-01 21:12:56.442369938,30.0,4.0,120.0,None
2020-09-01 21:12:59.490318060,30.0,4.0,120.0,None
2020-09-01 21:13:02.537092209,30.0,4.0,120.0,None


In [31]:
list(controller.log_to_df()['forecast1']['c'])[1:] == [40.0, 40.0]

True

In [ ]:
controller.query_control(time.time())
for df in controller.log_to_df().values():
    display(df)

In [ ]:
display(controller.running_controllers.__str__())
display(controller.executed_controllers.__str__())
display(controller.timeout_controllers.__str__())

In [ ]:
display(len(controller.log_to_df()['mpc1'].columns))
display(controller.running_controllers.__str__())
display(controller.executed_controllers.__str__())
display(controller.timeout_controllers.__str__())

In [ ]:
for i in range(3):
    controller.query_control(time.time())
    time.sleep(1)
for df in controller.log_to_df().values():
    display(df)

In [ ]:
display(len(controller.log_to_df()['mpc1'].columns))
display(controller.running_controllers.__str__())
display(controller.executed_controllers.__str__())
display(controller.timeout_controllers.__str__())